# Figure 6

## Imports

In [1]:
import dolfin
import numpy
import scipy.optimize

from scipy.interpolate import interp1d

import dolfin_mech                    as dmech
import matplotlib.pyplot              as plt
import micro_poro_structure_generator as gen

## Importing experimental data

In [2]:
jorba_tensile_test = numpy.load('jorba_tensile_test.npy')
eps_jorba_tensile_test = jorba_tensile_test[:, 0]
sig_jorba_tensile_test = jorba_tensile_test[:, 1]

## Defining geometry

In [ ]:
seeds_filename    = "Fig6.dat"
mesh_filebasename = "Fig6-mesh"
qois_filename     = "Fig6-qois.dat"
res_basename      = "Fig6"

domain_y = 0.1 * 0.8
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.012 * 0.8

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = True)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

## Loading 

In [4]:
load_params = {}
load_params["pf"] = 0
load_params["sigma_bar_00"] = 1
load_params["sigma_bar_11"] = 0.0
load_params["sigma_bar_01"] = 0.0
load_params["sigma_bar_10"] = 0.0

## Identifier function

In [5]:
def paramter_identifier(exp_data, params_initial, bnds):

    eps_jorba_tensile_test = exp_data[:, 0]
    sig_jorba_tensile_test = exp_data[:, 1]

    def J_cost(params_dimless):
        par = [params_initial[0]*(1 + params_dimless[0]), params_initial[1]*(1 + params_dimless[1]), params_initial[2]*(1 + params_dimless[2]), params_initial[3]*(1 + params_dimless[3])]
        print(par)

        mat_params = {"model":"exponential", "parameters":{"beta1":par[0], "beta2":par[1], "beta3":par[2], "alpha":par[3]}} 


        dmech.run_HollowBox_MicroPoroHyperelasticity(
            dim=2,
            mesh=mesh,
            mat_params=mat_params,
            load_params=load_params,
            step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.005, "dt_max":0.1},
            res_basename=res_basename,
            write_qois_limited_precision=False,
            verbose=1
        )

        qois_vals = numpy.loadtxt(qois_filename)
        qois_name_list = open(qois_filename).readline().split()
        sigma_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
        eps_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]


        model_interpolator = interp1d(sigma_lst, eps_lst, kind='cubic')  

        JC = 0
        for i in range(len(exp_data)):
            JC += (eps_jorba_tensile_test[i] - model_interpolator(sig_jorba_tensile_test[i]))**2

        print("JC: " +str(JC))
        return JC


    params_ini_dimless = [0, 0,  0, 0]
    params_id= (scipy.optimize.minimize(J_cost, x0=params_ini_dimless, method="TNC", bounds=bnds, tol=1e-5)).x

    params_id = [params_initial[0]*(1 + params_id[0]), params_initial[1]*(1 + params_id[1]), params_initial[2]*(1 + params_id[2]), params_initial[3]*(1 + params_id[3])]

    return params_id

In [ ]:
params_initial = [1.3655572609573783, 0.13038534018718229, 0.7539299887370342, 2.5] 
bnds = [(-0.9, 10), (-0.8, 10), (-0.2, 0.2), (-0.2, 4)]
params_id_jorba =  paramter_identifier(jorba_tensile_test, params_initial, bnds)

## Model response 

In [7]:
def graph_printer_jorba(params):
    mat_params = {"model":"exponential", "parameters":{"beta1":params[0], "beta2":params[1], "beta3":params[2], "alpha":params[3]}}

    dmech.run_HollowBox_MicroPoroHyperelasticity(
        dim=2,
        mesh=mesh,
        mat_params=mat_params,
        load_params=load_params,
        step_params={"Deltat":1., "dt_ini":0.1, "dt_min":0.005, "dt_max":0.1},
        res_basename=res_basename,
        write_qois_limited_precision=False,
        verbose=1
    )

    qois_vals = numpy.loadtxt(qois_filename)
    qois_name_list = open(qois_filename).readline().split()
    sigma_lst = qois_vals[:, qois_name_list.index("sigma_bar_XX") - 1]
    eps_lst = qois_vals[:, qois_name_list.index("U_bar_XX") - 1]

    plt.plot()
    plt.rc('xtick' , labelsize=14)
    plt.rc('ytick' , labelsize=14)
    plt.rc('legend', fontsize=12)
    plt.xlabel(r'$\epsilon_x~()$', fontsize=16)
    plt.ylabel(r'$\bar{\sigma}_x~(kPa)$', fontsize=16)


    plt.plot(eps_jorba_tensile_test, sig_jorba_tensile_test, '#D94801', label='[Jorba et al, 2019]')

    plt.plot(eps_lst, sigma_lst, '#084594', label='Model, $\gamma = 0~dyn/cm$')

    plt.xlim(0, 0.35)
    plt.ylim(0, 1)
    plt.legend(loc = 'lower right', fontsize=12, shadow=True)
    plt.savefig('Jorba_iden.pdf',bbox_inches='tight')
    plt.show()

### Plot

In [ ]:
graph_printer_jorba(params_id_jorba)